<a href="https://colab.research.google.com/github/gustavoborgesguimaraes/python-projects/blob/main/desafio_autoglass_nivelamento.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Autor: Gustavo Borges Guimarães

# Desafio: Consumo de Dados para Previsão do Tempo dos Municípios do ES.

## Objetivo

Avaliar conhecimentos nas linguagens Python e SQL e na engine de processamento Apache Spark.

## Descrição

Neste desafio, você desenvolverá um notebook que será responsável por extrair dados de previsão do tempo dos municípios do ES. Para consultar todas as cidades dessa região, utilizaremos a API do IBGE. No caso, basta realizar uma requisição HTTP com o método GET, utilizando a URL abaixo:

```
https://servicodados.ibge.gov.br/api/v1/localidades/estados/32/municipios
```

Com esses dados, gerar um data frame e a partir dele uma temp view. Ex: "cities"

Utilizando os nomes das cidades, deverão ser consultados os dados de previsão de tempo para cada município. Para realizar essa consulta, poderá ser utilizada qualquer uma das APIs informadas no link abaixo.

[Public APIs - Wather](https://github.com/public-apis/public-apis#weather)

Obs.: Para algumas, pode ser necessário cadastrar-se para acessar sua API Key. Mas nenhuma delas deve precisar cadastrar cartão de crédito ou adicionar qualquer valor monetário para utilizar. Caso alguma solicite, basta optar por outra.

Com os dados consultados, gerar um data frame e partir dele outra temp view. Ex: "forecasts"

Com as temp views geradas, utilizar Spark SQL para criar queries e gerar data frames das seguintes tabelas:

- Tabela 1: dados de previsão do tempo para os próximos cinco dias, para cada data e cidade consultadas. As colunas dessa tabela serão:
    - Cidade
    - CodigoDaCidade
    - Data
    - Regiao
    - Pais
    - Latitude
    - Longigute
    - TemperaturaMaxima
    - TemperaturaMinima
    - TemperaturaMedia
    - VaiChover
    - ChanceDeChuva
    - CondicaoDoTempo
    - NascerDoSol
    - PorDoSol
    - VelocidadeMaximaDoVento
    
    Obs.: Os valores da coluna "VaiChover" deverá ser "Sim" ou "Não". E a coluna "CodigoDaCidade" é o ID retornado junto com os nomes da cidades na API do IBGE.
    Obs.: Dependendo da API utilizada, algumas colunas podem não existir e ficarão em branco. Você deve optar por uma API que traga o maior número de informações possível.

- Tabela 2: quantidade de dias com chuva e sem chuva para os dias consultados, para cada data consultada. Colunas:
    - Cidade
    - QtdDiasVaiChover
    - QtdDiasNaoVaiChover
    - TotalDiasMapeados

Essas tabelas deverão ser exportadas em formado CSV e entregue no final do desafio.

## To Do

[ ] - Consultar municípios do ES, gerar um data frame e criar uma temp view com esses dados.
[ ] - Consultar dados do tempo para cada município, gerar um data frame e criar uma outra temp view.
[ ] - Utilizar Spark SQL para gerar os data frames das Tabelas 1 e 2.
[ ] - Exportar os data frames para CSV.

## Atenção

- Existe um limite de requisições de 10000 requests por conta cadastrada na m3o.
- Essa API pode retornar cidades de outras regiões que possuem nome semelhante a alguma cidade do ES. Pode mantê-las ou filtrar para gerar as tabelas apenas com dados de Regiao = Vila Velha. Fica a seu critério.

## Entregando o desafio

Concluindo todos os passos informados em To Do, basta salvar o arquivo .ipynb do notebook e zipar juntamente com os CSVs das duas tabelas e postar no grupo do Teams.



PREPA-----------------------------------------------
### Caso queira utilizar o colab do google para executar o script Spark descomente as linhas abaixo

In [ ]:
#PREPARAÇÃO DO AMBIENTE.

#Instalando o JDK.
!apt-get install openjdk-11-jdk-headless -qq > /dev/null

#Baixando o Apache Hadoop.
!wget -q https://dlcdn.apache.org/spark/spark-3.3.1/spark-3.3.1-bin-hadoop3.tgz

#Descompactando o Apache Hadoop.
!tar xf spark-3.3.1-bin-hadoop3.tgz

#Definindo as variáveis de ambiente do Java e do Spark.
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.3.1-bin-hadoop3"

#Baixando o ngrok.
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip

#Descompactando o ngrok.
!unzip ngrok-stable-linux-amd64.zip

#Definindo a URL local para a consumo da API.
get_ipython().system_raw('./ngrok http 4050 &')
!curl -s http://localhost:4040/api/tunnels

#Instalando o FindSpark.
!pip install -q findspark

#Instalando o UniDecode.
!pip install -q unidecode

#Importando os módulos/libs que serão usados.
import findspark
findspark.init()
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark import SparkContext
import requests
import json
import unidecode
import pandas as pd

#Buildando o Spark Session
spark = SparkSession.builder \
        .master("local[1]") \
        .appName("SparkbyExamples.com") \
        .config('spark.ui.port', '4050') \
        .getOrCreate()

In [2]:
#Constante da Unidade Federativa do ES
#Para mais, consulte em "servicodados.ibge.gov.br"
CodUF = '32'

#Endereço da API do IBGE
UrlDaApi = f"https://servicodados.ibge.gov.br/api/v1/localidades/estados/{CodUF}/municipios"

#Efetuando a requisição na API do IBGE
RequestDaApi = requests.get(UrlDaApi)

In [ ]:
#Convertento o retorno da API em JSON
JsonObject = RequestDaApi.json()

#OPCIONAL: Criando um arquivo JSON local.
with open('JsonFileResponse.json', 'w') as json_file:
  json.dump(JsonObject, json_file, indent=4)

#OPCIONAL: Criando um arquivo Parquet local.
  #Código...

#Criando o DataFrame GERAL do Objeto Json
df = spark.createDataFrame(JsonObject)

#Criando um DataFrame de cidades com colunas específicas.
df_cidades = df.withColumn("CodCidade", F.col("id"))\
               .withColumn("NomeCidade", F.col("nome"))\
               .drop("id", "microrregiao", "nome", "regiao-imediata")

#Contando o número de registros do DataFrame Cidades
QtdRowsDataFrameCidades = df.count()
print('Existem ' +  str(QtdRowsDataFrameCidades) + ' linhas no DataFrame Cidades')

#Visualizando todos os registros do Data frame Cidades
df_cidades.show(QtdRowsDataFrameCidades)

In [4]:
#Criando a TempView "cities"
df_cidades.createOrReplaceTempView("cities")

#Testando a consulta SparkSQL na TempView
VarCity = spark.sql('select * from cities')

#Criando a coleção de cidades para usar na API de previsões
CityCollect = VarCity.collect()

In [22]:
#Criando o Data Frame das previsões do tempo.
dfPrevTempo = pd.DataFrame(columns= ['Cidade', 'CodigoDaCidade', 'Pais', 'Latitude',
                                     'Longitude', 'NascerDoSol', 'PorDoSol', 'Populacao',
                                     'Data', 'TemperaturaMaxima', 'TemperaturaMinima',
                                     'SensacaoTermica', 'CondicaoDoTempo', 'VelocidadeDoVento'])

#Constantes p/uso na API
MyAPIKey = "d15318e0f35125eb2040ecd74952410a"
StateCod = "ES"
CountryCod = "BR"

for cid in CityCollect:
  Link = f"https://api.openweathermap.org/data/2.5/forecast?q={cid['NomeCidade']},{StateCod},{CountryCod}&appid={MyAPIKey}&lang=pt_br"
  Requisicao = requests.get(Link)
  RequisicaoDic = Requisicao.json()
  if RequisicaoDic['cod'] == '200':
    Cidade = (RequisicaoDic['city']['name']) #Cidade
    CodigoDaCidade = (RequisicaoDic['city']['id']) #CodigoDaCidade
    Pais = (RequisicaoDic['city']['country']) #País
    Latitude = (RequisicaoDic['city']['coord']['lat']) #Latitude
    Longitude = (RequisicaoDic['city']['coord']['lon']) #Longitude
    NascerDoSol = (RequisicaoDic['city']['sunrise']) #NascerDoSol
    PorDoSol = (RequisicaoDic['city']['sunset']) #PorDoSol
    Populacao = (RequisicaoDic['city']['population']) #População
    for CidDesc in RequisicaoDic['list']:
      Data = (CidDesc['dt_txt']) #Data
      TemperaturaMaxima = (CidDesc['main']['temp_max'] -273.15) #TemperaturaMáxima
      TemperaturaMinima = (CidDesc['main']['temp_min'] -273.15) #TemperaturaMínima
      SensacaoTermica = (CidDesc['main']['feels_like'] -273.15) #SensaçãoTérmica
      CondicaoDoTempo = (CidDesc['weather'][0]['description']) #CondicaoDoTempo
      VelocidadeDoVento = (CidDesc['wind']['speed']) #VelocidadeDoVento
      #Inserindo os dados no DataFrame de Previsões do tempo
      dfPrevTempo = dfPrevTempo.append({'Cidade' : Cidade, 'CodigoDaCidade' : CodigoDaCidade, 'Pais' : Pais, 'Latitude' : Latitude,
                                        'Longitude' : Longitude, 'NascerDoSol' : NascerDoSol, 'PorDoSol' : PorDoSol, 'Populacao' : Populacao,
                                        'Data' : Data, 'TemperaturaMaxima' : TemperaturaMaxima, 'TemperaturaMinima' : TemperaturaMinima,
                                        'SensacaoTermica' : SensacaoTermica, 'CondicaoDoTempo' : CondicaoDoTempo, 'VelocidadeDoVento' : VelocidadeDoVento},
                                        ignore_index = True)


In [23]:
dfPrevTempo

,Cidade,CodigoDaCidade,Pais,Latitude,Longitude,NascerDoSol,PorDoSol,Populacao,Data,TemperaturaMaxima,TemperaturaMinima,SensacaoTermica,CondicaoDoTempo,VelocidadeDoVento
0,Afonso Cláudio,3473129,BR,-20.0742,-41.1239,1674807912,1674854936,13543,2023-01-27 18:00:00,25.61,25.26,26.52,chuva moderada,2.24
1,Afonso Cláudio,3473129,BR,-20.0742,-41.1239,1674807912,1674854936,13543,2023-01-27 21:00:00,24.17,23.37,25.04,chuva moderada,1.13
2,Afonso Cláudio,3473129,BR,-20.0742,-41.1239,1674807912,1674854936,13543,2023-01-28 00:00:00,21.42,21.42,22.12,chuva leve,1.05
3,Afonso Cláudio,3473129,BR,-20.0742,-41.1239,1674807912,1674854936,13543,2023-01-28 03:00:00,21.00,21.00,21.69,nublado,0.97
4,Afonso Cláudio,3473129,BR,-20.0742,-41.1239,1674807912,1674854936,13543,2023-01-28 06:00:00,20.55,20.55,21.25,chuva leve,0.93
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2795,Vitória,3444924,BR,-20.3194,-40.3378,1674807700,1674854770,312656,2023-02-01 03:00:00,23.50,23.50,24.20,nublado,3.27
2796,Vitória,3444924,BR,-20.3194,-40.3378,1674807700,1674854770,312656,2023-02-01 06:00:00,22.93,22.93,23.58,nuvens dispersas,3.16
2797,Vitória,3444924,BR,-20.3194,-40.3378,1674807700,1674854770,312656,2023-02-01 09:00:00,22.67,22.67,23.29,nublado,3.05
2798,Vitória,3444924,BR,-20.3194,-40.3378,1674807700,1674854770,312656,2023-02-01 12:00:00,26.68,26.68,28.33,nublado,3.48


-------------------------------------------------